Для домашнего задания я выбрала **роман Набокова "Приглашение на казнь"**.

#### 1. Убираем из текста знаки препинания и нумерацию глав (римскими цифрами) с помощью регулярных выражений:

In [136]:
import re

with open("novel.txt", "r", encoding = "utf-8") as file:
    text = file.read()
    # убираем знаки препинания:
    text = re.sub(r'[^\w\s]', '', text) 
    # убираем нумерацию глав:
    text = re.sub(r'\n[A-Z]{1,5}\n\n\n','', text)

#### 2. Обрабатываем текст через mystem 

In [137]:
from pymystem3 import Mystem
m = Mystem()

# лемматизируем текст
lemmas = m.lemmatize(text) 

# делаем из lemmas сплошной текст вместо списка
lemmas_final = ''.join(lemmas)

# сохраняем в файл 

with open("mystem_result","w", encoding = "utf-8") as file:
    file.write(lemmas_final)

#### 3. Токенизируем текст через nltk, обрабатываем через Pymorphy и сохраняем в json

In [138]:
import nltk
from nltk.tokenize import word_tokenize
import json
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()

# токенизируем текст:
tokens = word_tokenize(text)

# создаем jsonl файл
with open('pymorphy_result.jsonl', 'a', encoding='utf-8') as file:
    
# проходя по циклу дополняем файл новым словарем для каждого токена)

    for token in tokens:
        ana = morph.parse(token)[0]
        ana_dict = {}
        ana_dict["lemma"] = ana.normal_form
        ana_dict["word"] = ana.word
        ana_dict["pos"] = ana.tag.POS
        json.dump(ana_dict, file, ensure_ascii = False)
        file.write("\n")
        

#### 4. Вопросы

**1. Какую долю слов составляет каждая часть речи?**

In [147]:
# словарь для счета кол-ва слов определенной
# части речи
pos_dict = {
    'NOUN':0,
    'ADJF':0,
    'ADJS':0,
    'COMP':0,
    'VERB':0,
    'INFN':0,
    'PRTF':0,
    'PRTS':0,
    'GRND':0,
    'NUMR':0,
    'ADVB':0,
    'NPRO':0,
    'PRED':0,
    'PREP':0,
    'CONJ':0,
    'PRCL':0,
    'INTJ':0
}

# проходим по каждому токену 
for token in tokens:
    ana = morph.parse(token)[0]
# обновляем счетчик частей речи в словаре pos_dict
    if ana.tag.POS in pos_dict.keys():
        pos_dict[ana.tag.POS] += 1

print ("Доля каждой из частей речи в тексте: \n")
for key in pos_dict.keys():
       print(key, " - ", pos_dict[key]/len(tokens))

Доля каждой из частей речи в тексте: 

NOUN  -  0.258069270449521
ADJF  -  0.11748956030459347
ADJS  -  0.009334315892901008
COMP  -  0.00402849422746254
VERB  -  0.13674772783099975
INFN  -  0.02117415868337018
PRTF  -  0.01714566445590764
PRTS  -  0.005281257676246622
GRND  -  0.020953082780643575
NUMR  -  0.0039056742815033164
ADVB  -  0.08128224023581429
NPRO  -  0.06644559076394006
PRED  -  0.00449521002210759
PREP  -  0.10002456398919185
CONJ  -  0.09882092851879146
PRCL  -  0.04890690248096291
INTJ  -  0.002112503070498649


**2. Топ-20 (по частотности) глаголов и наречий.**

In [145]:
# функция, которая помогает вывести топ-20 наречий и глаголов
# из уже готовых, отсортированных словарей частотности 
# различных наречий и глаголов в тексте

def making_top(dictionary):
    counter = 0
    for key in dictionary.keys():
        print(key, dictionary[key])
        counter += 1
        if counter == 20:
              break


verb_dict = {
}
adverb_dict = {
}

for token in tokens:
    ana = morph.parse(token)[0]
    if ana.tag.POS == "VERB":
        if ana.normal_form not in verb_dict.keys():
            verb_dict[ana.normal_form] = 1
        else:
            verb_dict[ana.normal_form] += 1
            
    elif ana.tag.POS == "ADVB":
        if ana.normal_form not in adverb_dict.keys():
            adverb_dict[ana.normal_form] = 1
        else:
            adverb_dict[ana.normal_form] += 1

#сортировка словарей по значению в порядке убывания
sorted_verb_dict = dict(sorted(verb_dict.items(), key=lambda item: item[1], reverse=True))
sorted_adverb_dict = dict(sorted(adverb_dict.items(), key=lambda item: item[1], reverse=True))

#вывод результатов
print("Топ-20 глаголов:")
results = making_top(sorted_verb_dict)

print("\n\n")

print("Топ-20 наречий:")
results = making_top(sorted_adverb_dict)


Топ-20 глаголов:
быть 423
сказать 204
мочь 121
знать 107
хотеть 62
говорить 57
казаться 43
спросить 38
продолжать 38
видеть 36
сидеть 32
смотреть 32
стоять 30
идти 29
понимать 28
выйти 26
стать 26
проговорить 26
просить 26
начать 25



Топ-20 наречий:
ещё 160
уже 111
только 110
тут 93
опять 76
теперь 76
где 69
ничего 62
там 58
вдруг 53
сейчас 47
очень 42
потом 38
совсем 36
наконец 34
тогда 32
сразу 28
тихо 27
затем 27
несколько 25


#### 5. Посмотрите документацию для nltk н-грамм (nltk.bigrams, например), попробуйте составить топ-25 биграмм и триграмм для вашего текста в лемматизированном виде (только леммы, без знаков препинания). Почему получаются именно такие?

In [133]:
from nltk.util import ngrams

#функция создает словарь со всеми н-граммами в тексте и подсчитывает их частотность
def making_ngrams_dict(lemmas_text, n):
    ngrams_dict = {
    }
    ngrams_list = ngrams(lemmas_text.split(), n)
    for ngram in ngrams_list:
        if ngram not in ngrams_dict.keys():
            ngrams_dict[ngram] = 1
        else:
            ngrams_dict[ngram] += 1
    return ngrams_dict

#функция сортирует словарь по значениям от большего к меньшему
def sorting_dict(ngrams_dict):
    sorted_ngrams_dict = dict(sorted(ngrams_dict.items(), key=lambda item: item[1], reverse=True))
    return sorted_ngrams_dict

#функция выводит топ-25 н-грамм
def making_top(sorted_ngrams_dict):
    counter = 0
    for key in sorted_ngrams_dict.keys():
        print(key, " - ", sorted_ngrams_dict[key])
        counter += 1
        if counter == 25:
              break

            
#используем лемматизированный текст
with open("mystem_result.txt", "r", encoding="utf-8") as file:
    lemmas_text = file.read()
    
    #частотность биграмм
    bigram_dict = making_ngrams_dict(lemmas_text, 2)
    sorted_bigram_dict = sorting_dict(bigram_dict)
    print("Топ-25 биграмм в тексте:")
    result = making_top(sorted_bigram_dict)
    
    print("\n\n")
    
    #частотность триграмм
    trigram_dict = making_ngrams_dict(lemmas_text, 3)
    sorted_trigram_dict = sorting_dict(trigram_dict)
    print("Топ-25 триграмм в тексте:")
    result = making_top(sorted_trigram_dict)
 

Топ-25 биграмм в тексте:
('мсье', 'пьер')  -  201
('родрига', 'иванович')  -  55
('сказать', 'цинциннат')  -  46
('не', 'мочь')  -  36
('мочь', 'быть')  -  35
('то', 'что')  -  35
('что', 'я')  -  34
('сказать', 'мсье')  -  34
('ничто', 'не')  -  33
('то', 'быть')  -  31
('и', 'вот')  -  31
('я', 'не')  -  30
('и', 'с')  -  28
('не', 'знать')  -  28
('к', 'цинциннат')  -  28
('на', 'стол')  -  27
('у', 'я')  -  27
('как', 'бы')  -  26
('цинциннат', 'и')  -  25
('так', 'что')  -  25
('спрашивать', 'цинциннат')  -  25
('у', 'он')  -  24
('это', 'быть')  -  24
('и', 'не')  -  23
('он', 'и')  -  23



Топ-25 триграмм в тексте:
('сказать', 'мсье', 'пьер')  -  34
('мсье', 'пьер', 'и')  -  20
('во', 'всякий', 'случай')  -  10
('на', 'самый', 'дело')  -  8
('и', 'мсье', 'пьер')  -  8
('продолжать', 'мсье', 'пьер')  -  8
('я', 'казаться', 'что')  -  7
('о', 'то', 'что')  -  7
('садиться', 'за', 'стол')  -  6
('в', 'конец', 'конец')  -  6
('сказать', 'цинциннат', 'и')  -  6
('с', 'тот', 'же')  -



Получаются такие биграммы и триграммы потому, что главными героем текста, вокруг которых строится всё повествование, являются Цинциннат, Родрига Иванович и мьсе Пьер. Их имена чаще всего встречаются на страницах книги в виде н-грамм или в составе н-грамм. 

Кроме того сюжет во многом строится на диалогах, размышлениях: это объясняет большую частотность н-грамм со словами 'знать', 'говорить', 'садиться'.

#### 6. Возьмите абзац из изначального текста (3 - 8 предложений), замените в нём время всех глаголов (например, прошедшее на настоящее или будущее), число всех существительных (единственное на множественное, множественное на единственное), сделайте согласование по числу глаголов и существительных.

In [135]:
paragraph = "Он  задумался.  Потом  бросил  карандаш,  встал, \
заходил. Донесся  бой  часов. Пользуясь  их  звоном,  как  \
платформой, поднялись на поверхность шаги; платформа уплыла, \
шаги остались, и вот в камеру вошли:  Родион с супом и \
господин библиотекарь с каталогом."

new_paragraph = ""
words = word_tokenize(paragraph)

for i, word in enumerate(words):
    ana = morph.parse(word)[0]
    
    if 'NOUN' in ana.tag:
        if ana.tag.number == "sing":
            form = ana.inflect({'plur'})
        elif ana.tag.number == "plur":
            form = ana.inflect({'sing'})
        
        new_paragraph = new_paragraph + form[0] + " "
        
    elif 'VERB' in ana.tag:
        if ana.tag.tense == "past":
            if ana.tag.aspect == "perf":
                pre_form = ana.inflect({'futr','3per'})
            elif ana.tag.aspect == "impf":
                pre_form = ana.inflect({'pres','3per'})
        elif ana.tag.tense in ["pres", "futr"]:
            pre_form = ana.inflect({'past'})
        
        if ana.tag.number == "sing":
            form = pre_form.inflect({'plur'})
        elif ana.tag.number == "plur":
            form = pre_form.inflect({'sing'})
    
        new_paragraph = new_paragraph + form[0] + " "
        
    else:
        new_paragraph = new_paragraph + word + " "

new_paragraph = re.sub(r'\s+(?=(?:[,.?!:;…]))','', new_paragraph)
print(new_paragraph)


Он задумаются. Потом бросят карандаши, встанут, заходят. донесутся бои часа. Пользуясь их звонами, как платформами, поднимется на поверхности шаг; платформы уплывут, шаг останется, и вот в камеры войдёт: родионы с супами и господа библиотекари с каталогами. 


При написании этого кода моя логика была такова: при изменении числа одновременно и существительного, и глагола, согласование по числу произойдет само собой. Это связано с особенностями отрывка текста, который я выбрала для задания. Однако, этот подход не учитывает наличие двух подлежащих (как в последнем предложении) и, как следствие, необходимость сохранения множественного числа глагола даже при изменении числа каждого из подлежащих.

Кроме того, в результате изменения числа возникают конструкции, разительно отличающиеся по смыслу от конструкций в исходном тексте: бои часа - это не множественное число 'боя часов'.

Как видно из получившегося текста, для создания грамматичного текста в результате изменениея граммем недостаточно программ, анализирующих морфологию. Важно учитывать синтаксис в том числе.